<a href="https://colab.research.google.com/github/xiulinyang/Aspect_Detection/blob/main/GPT2_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 33.6 MB/s 
     |████████████████████████████████| 596 kB 50.3 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [4]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
from random import sample
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import re

Two questions: (1) should we stem the label before training? (2) how should we get the ids of the labels

In [5]:
def template(sentence, aspects=None):
    if aspects is None:
        tem = f" Sentence: {sentence} Aspect: "
    else:
        tem = f" Sentence: {sentence} Aspect: {aspects}."
    return tem

def generateTemplate(data):
    train_idxs = np.random.randint(0, len(data) - 1, 3)
    for i, sent in enumerate(data):
      text = ""
      for ti in train_idxs:
          sentence = data.iloc[ti]["sentence"]
          sentence = re.sub('[^A-Za-z0-9]+', ' ', sentence).lower()
          aspects = ", ".join(data.iloc[ti]["aspect_pos_string"])
#         aspects = train_df.iloc[ti]["topic"]
          text += template(sentence, aspects)
    
          text += template(data.iloc[i]["sentence"])
          aspects = ", ".join(data.iloc[i]["aspect_pos_string"])
#     aspects = test_df.iloc[test_idxs]["topic"]
    return text, aspects

In [6]:
train = pd.read_json('/content/gdrive/MyDrive/in_topic/train.jsonl', lines =True)
dev = pd.read_json('/content/gdrive/MyDrive/in_topic/dev.jsonl', lines =True)
test= pd.read_json('/content/gdrive/MyDrive/in_topic/test.jsonl', lines =True)

In [102]:
class ArgumentData:
  def __init__(self, tokenizer, data, max_length=512):
    self.input_ids =[]
    self.attention_mask =[]
    self.aspect =[]
    for i, sent in enumerate(data['sentence']):
      train_idxs = np.random.randint(0, len(data)-1, 3)
      text = ""
      for ti in train_idxs:
          sentence = data.iloc[ti]["sentence"]
          sentence = re.sub('[^A-Za-z0-9]+', ' ', sentence).lower()
          aspects = ", ".join(data.iloc[ti]["aspect_pos_string"])
          text += template(sentence, aspects)
    
      text += template(data.iloc[i]["sentence"], ', '.join(data.iloc[i]['aspect_pos_string']))
      encoding_dic = tokenizer(text, truncation=True, max_length=max_length, padding='max_length')
      self.input_ids.append(torch.tensor(encoding_dic['input_ids']))
      self.attention_mask.append(torch.tensor(encoding_dic['attention_mask']))
      self.aspect.append(aspects)
  def __len__(self):
    return len(self.input_ids)
  
  def __getitem__(self, idx):
    return self.input_ids[idx], self.attention_mask[idx], self.aspect[idx]

In [103]:
train_dir = '/content/gdrive/MyDrive/in_topic/train.jsonl'
dev_dir = '/content/gdrive/MyDrive/in_topic/dev.jsonl'
test_dir ='/content/gdrive/MyDrive/in_topic/test.jsonl'
def data_load(dir, tokenizer):
  data = pd.read_json(dir, lines=True)
  processed_data = ArgumentData(tokenizer,data,max_length=512)
  return processed_data

In [104]:
model_name ='gpt2'
torch.manual_seed(123)
tokenizer =GPT2Tokenizer.from_pretrained(model_name, bos_token='<startoftext>', eos_token ='<endoftext>', pad_token='<pad>')
model = GPT2LMHeadModel.from_pretrained(model_name).cuda()
model.resize_token_embeddings(len(tokenizer))


loading file https://huggingface.co/gpt2/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/684fe667923972fb57f6b4dcb61a3c92763ad89882f3da5da9866baf14f2d60f.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f
loading file https://huggingface.co/gpt2/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/c0c761a63004025aeadd530c4c27b860ec4ecbe8a00531233de21d865a402598.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/gpt2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/gpt2/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/gpt2/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb1993821

Embedding(50260, 768)

In [105]:
train_dataset = data_load(train_dir, tokenizer=tokenizer)
test_dataset = data_load(test_dir, tokenizer)
dev_dataset = data_load(dev_dir, tokenizer)



In [138]:
train_dataset = train_dataset

In [106]:
from transformers import Trainer, TrainingArguments
torch.cuda.empty_cache()

In [139]:
training_args= TrainingArguments(output_dir='/content/gdrive/MyDrive/in_topic/results', num_train_epochs=2, logging_steps=100,
                                  save_strategy='epoch', 
                                 per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                 warmup_steps=100, weight_decay=0.01, logging_dir='/content/gdrive/MyDrive/in_topic/logs')
Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=dev_dataset, 
        data_collator=lambda data:{'input_ids': torch.stack([f[0] for f in data]),
                                   'attention_mask': torch.stack([f[1] for f in data]),
                                   'labels': torch.stack([f[0] for f in data])}).train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


RuntimeError: ignored

In [141]:
from nltk.translate.bleu_score import sentence_bleu

In [135]:
model.eval()
test_dataset = pd.read_json(test_dir, lines=True)

test_dt =[]
test_aspect =[]
for i, sent in enumerate(test_dataset['sentence']):
      train_idxs = np.random.randint(0, len(test_dataset)-1, 3)
      text = ""
      for ti in train_idxs:
          sentence = test_dataset.iloc[ti]["sentence"]
          sentence = re.sub('[^A-Za-z0-9]+', ' ', sentence).lower()
          aspects = ", ".join(test_dataset.iloc[ti]["aspect_pos_string"])
          text += template(sentence, aspects)
      text += template(test_dataset.iloc[i]["sentence"])
      test_dt.append(text)
      tst_aspect =', '.join(test_dataset.iloc[i]['aspect_pos_string'])
      test_aspect.append(tst_aspect)

scores=[]
for text, label in tqdm(zip(test_dt, test_aspect)):
  prompt = text
  generated = tokenizer(f'<startoftext>{prompt}', return_tensors='pt').input_ids.cuda()
  sample_outputs = model.generate(generated, do_sammple=False, top_k=5, max_length=512, top_p = 0.5, temperature=0.9, num_return_sequences=1)
  predicted_text = tokenizer.decode(sample_outputs[0], skip_special_token=True)

  newtext = predicted_text.replace(test_dt, "")
  newaspect = newtext.split(".")[0]
  score = sentence_bleu(aspect.split(), newaspect.split())
  scores.append(score)

 

0it [00:00, ?it/s]


RuntimeError: ignored